# The Gauss-Seidel Method

## Introduction

The **Gauss-Seidel method** is an iterative technique for solving systems of linear equations of the form:

$$\mathbf{A}\mathbf{x} = \mathbf{b}$$

where $\mathbf{A} \in \mathbb{R}^{n \times n}$ is a coefficient matrix, $\mathbf{x} \in \mathbb{R}^n$ is the unknown vector, and $\mathbf{b} \in \mathbb{R}^n$ is the constant vector.

## Mathematical Foundation

### Matrix Decomposition

The Gauss-Seidel method decomposes the matrix $\mathbf{A}$ into:

$$\mathbf{A} = \mathbf{L} + \mathbf{D} + \mathbf{U}$$

where:
- $\mathbf{L}$ is the strictly lower triangular part of $\mathbf{A}$
- $\mathbf{D}$ is the diagonal of $\mathbf{A}$
- $\mathbf{U}$ is the strictly upper triangular part of $\mathbf{A}$

### Iteration Formula

The iterative scheme is:

$$(\mathbf{L} + \mathbf{D})\mathbf{x}^{(k+1)} = \mathbf{b} - \mathbf{U}\mathbf{x}^{(k)}$$

Component-wise, this becomes:

$$x_i^{(k+1)} = \frac{1}{a_{ii}}\left(b_i - \sum_{j=1}^{i-1} a_{ij}x_j^{(k+1)} - \sum_{j=i+1}^{n} a_{ij}x_j^{(k)}\right)$$

The key distinction from the Jacobi method is that Gauss-Seidel uses the **most recently computed values** $x_j^{(k+1)}$ for $j < i$ within the same iteration.

### Convergence Criteria

The method converges if any of the following conditions hold:

1. **Strict diagonal dominance**: $|a_{ii}| > \sum_{j \neq i} |a_{ij}|$ for all $i$
2. **Symmetric positive definite**: $\mathbf{A} = \mathbf{A}^T$ and all eigenvalues are positive
3. **Spectral radius**: $\rho(\mathbf{G}) < 1$ where $\mathbf{G} = -(\mathbf{L}+\mathbf{D})^{-1}\mathbf{U}$ is the iteration matrix

### Error Analysis

The error at iteration $k$ is:

$$\mathbf{e}^{(k)} = \mathbf{x}^{(k)} - \mathbf{x}^*$$

where $\mathbf{x}^*$ is the true solution. The error propagates as:

$$\mathbf{e}^{(k+1)} = \mathbf{G}\mathbf{e}^{(k)}$$

The convergence rate is determined by $\rho(\mathbf{G})$, with faster convergence when $\rho(\mathbf{G}) \ll 1$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm, solve

def gauss_seidel(A, b, x0=None, tol=1e-10, max_iterations=1000):
    """
    Solve the system Ax = b using the Gauss-Seidel method.
    
    Parameters:
    -----------
    A : ndarray
        Coefficient matrix (n x n)
    b : ndarray
        Right-hand side vector (n,)
    x0 : ndarray, optional
        Initial guess (default: zeros)
    tol : float
        Convergence tolerance
    max_iterations : int
        Maximum number of iterations
    
    Returns:
    --------
    x : ndarray
        Solution vector
    residuals : list
        History of residual norms
    iterations : int
        Number of iterations performed
    """
    n = len(b)
    x = x0.copy() if x0 is not None else np.zeros(n)
    residuals = []
    
    for iteration in range(max_iterations):
        x_old = x.copy()
        
        for i in range(n):
            # Sum of a_ij * x_j for j < i (using new values)
            sum_lower = np.dot(A[i, :i], x[:i])
            # Sum of a_ij * x_j for j > i (using old values)
            sum_upper = np.dot(A[i, i+1:], x_old[i+1:])
            # Update x_i
            x[i] = (b[i] - sum_lower - sum_upper) / A[i, i]
        
        # Compute residual norm
        residual = norm(b - A @ x)
        residuals.append(residual)
        
        # Check convergence
        if residual < tol:
            return x, residuals, iteration + 1
    
    return x, residuals, max_iterations

def is_diagonally_dominant(A):
    """Check if matrix A is strictly diagonally dominant."""
    n = A.shape[0]
    for i in range(n):
        diagonal = abs(A[i, i])
        off_diagonal_sum = sum(abs(A[i, j]) for j in range(n) if j != i)
        if diagonal <= off_diagonal_sum:
            return False
    return True

## Example 1: Diagonally Dominant System

Consider the system:

$$\begin{pmatrix} 4 & -1 & 0 & 0 \\ -1 & 4 & -1 & 0 \\ 0 & -1 & 4 & -1 \\ 0 & 0 & -1 & 3 \end{pmatrix} \begin{pmatrix} x_1 \\ x_2 \\ x_3 \\ x_4 \end{pmatrix} = \begin{pmatrix} 15 \\ 10 \\ 10 \\ 10 \end{pmatrix}$$

This matrix is diagonally dominant, guaranteeing convergence.

In [ ]:
# Define the diagonally dominant system
A1 = np.array([
    [4, -1, 0, 0],
    [-1, 4, -1, 0],
    [0, -1, 4, -1],
    [0, 0, -1, 3]
], dtype=float)

b1 = np.array([15, 10, 10, 10], dtype=float)

# Check diagonal dominance
print(f"Matrix is diagonally dominant: {is_diagonally_dominant(A1)}")

# Solve using Gauss-Seidel
x1, residuals1, iters1 = gauss_seidel(A1, b1, tol=1e-10)

# Compute exact solution for comparison
x_exact1 = solve(A1, b1)

print(f"\nGauss-Seidel solution: {x1}")
print(f"Exact solution: {x_exact1}")
print(f"Error norm: {norm(x1 - x_exact1):.2e}")
print(f"Iterations: {iters1}")

## Example 2: Larger System - Heat Equation Discretization

A common application is solving the 1D Poisson equation:

$$-\frac{d^2 u}{dx^2} = f(x)$$

Discretizing with finite differences yields a tridiagonal system with structure:

$$\frac{1}{h^2}\begin{pmatrix} 2 & -1 & & \\ -1 & 2 & -1 & \\ & \ddots & \ddots & \ddots \\ & & -1 & 2 \end{pmatrix}$$

In [ ]:
# Generate a tridiagonal system (1D Poisson equation)
n = 50
A2 = np.zeros((n, n))
for i in range(n):
    A2[i, i] = 2.0
    if i > 0:
        A2[i, i-1] = -1.0
    if i < n-1:
        A2[i, i+1] = -1.0

# Right-hand side (source term)
b2 = np.ones(n)

# Solve
x2, residuals2, iters2 = gauss_seidel(A2, b2, tol=1e-10)
x_exact2 = solve(A2, b2)

print(f"System size: {n}x{n}")
print(f"Error norm: {norm(x2 - x_exact2):.2e}")
print(f"Iterations: {iters2}")

## Comparison: Gauss-Seidel vs Jacobi Method

The Gauss-Seidel method typically converges faster than the Jacobi method because it uses updated values immediately. Let's compare their convergence rates.

In [ ]:
def jacobi(A, b, x0=None, tol=1e-10, max_iterations=1000):
    """
    Solve Ax = b using the Jacobi method for comparison.
    """
    n = len(b)
    x = x0.copy() if x0 is not None else np.zeros(n)
    residuals = []
    
    D_inv = 1.0 / np.diag(A)
    
    for iteration in range(max_iterations):
        x_new = np.zeros(n)
        for i in range(n):
            sum_off_diag = np.dot(A[i, :], x) - A[i, i] * x[i]
            x_new[i] = (b[i] - sum_off_diag) * D_inv[i]
        
        x = x_new
        residual = norm(b - A @ x)
        residuals.append(residual)
        
        if residual < tol:
            return x, residuals, iteration + 1
    
    return x, residuals, max_iterations

# Compare on the tridiagonal system
x_gs, res_gs, iters_gs = gauss_seidel(A2, b2, tol=1e-10)
x_jac, res_jac, iters_jac = jacobi(A2, b2, tol=1e-10)

print(f"Gauss-Seidel iterations: {iters_gs}")
print(f"Jacobi iterations: {iters_jac}")
print(f"Speedup factor: {iters_jac / iters_gs:.2f}x")

## Convergence Analysis and Visualization

Let's analyze and visualize the convergence behavior of both methods, and examine how the spectral radius affects convergence rate.

In [ ]:
def compute_iteration_matrix_gs(A):
    """Compute the Gauss-Seidel iteration matrix G = -(L+D)^{-1}U."""
    n = A.shape[0]
    L = np.tril(A, -1)
    D = np.diag(np.diag(A))
    U = np.triu(A, 1)
    LD_inv = np.linalg.inv(L + D)
    return -LD_inv @ U

def compute_iteration_matrix_jacobi(A):
    """Compute the Jacobi iteration matrix G = D^{-1}(L+U)."""
    D_inv = np.diag(1.0 / np.diag(A))
    L_plus_U = A - np.diag(np.diag(A))
    return -D_inv @ L_plus_U

# Compute spectral radii
G_gs = compute_iteration_matrix_gs(A2)
G_jac = compute_iteration_matrix_jacobi(A2)

rho_gs = max(abs(np.linalg.eigvals(G_gs)))
rho_jac = max(abs(np.linalg.eigvals(G_jac)))

print(f"Spectral radius (Gauss-Seidel): {rho_gs:.6f}")
print(f"Spectral radius (Jacobi): {rho_jac:.6f}")
print(f"\nTheoretical convergence rate ratio: {np.log(rho_jac)/np.log(rho_gs):.2f}x")

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Convergence comparison
ax1 = axes[0, 0]
ax1.semilogy(res_gs, 'b-', linewidth=2, label='Gauss-Seidel')
ax1.semilogy(res_jac, 'r--', linewidth=2, label='Jacobi')
ax1.set_xlabel('Iteration', fontsize=12)
ax1.set_ylabel('Residual Norm', fontsize=12)
ax1.set_title('Convergence Comparison', fontsize=14)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2: Solution profile
ax2 = axes[0, 1]
x_vals = np.linspace(0, 1, n)
ax2.plot(x_vals, x2, 'b-', linewidth=2, label='Gauss-Seidel')
ax2.plot(x_vals, x_exact2, 'k--', linewidth=1.5, label='Exact')
ax2.set_xlabel('x', fontsize=12)
ax2.set_ylabel('u(x)', fontsize=12)
ax2.set_title('Solution Profile (1D Poisson)', fontsize=14)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

# Plot 3: Spectral analysis - eigenvalues of iteration matrices
ax3 = axes[1, 0]
eig_gs = np.linalg.eigvals(G_gs)
eig_jac = np.linalg.eigvals(G_jac)
theta = np.linspace(0, 2*np.pi, 100)
ax3.plot(np.cos(theta), np.sin(theta), 'k-', linewidth=1, label='Unit circle')
ax3.scatter(eig_gs.real, eig_gs.imag, c='blue', s=30, alpha=0.6, label='Gauss-Seidel')
ax3.scatter(eig_jac.real, eig_jac.imag, c='red', s=30, alpha=0.6, label='Jacobi')
ax3.set_xlabel('Real', fontsize=12)
ax3.set_ylabel('Imaginary', fontsize=12)
ax3.set_title('Eigenvalues of Iteration Matrices', fontsize=14)
ax3.legend(fontsize=10)
ax3.set_aspect('equal')
ax3.grid(True, alpha=0.3)

# Plot 4: Convergence rate vs system size
ax4 = axes[1, 1]
sizes = [10, 20, 30, 50, 75, 100]
iters_gs_list = []
iters_jac_list = []

for size in sizes:
    # Generate tridiagonal system
    A_test = np.zeros((size, size))
    for i in range(size):
        A_test[i, i] = 2.0
        if i > 0:
            A_test[i, i-1] = -1.0
        if i < size-1:
            A_test[i, i+1] = -1.0
    b_test = np.ones(size)
    
    _, _, it_gs = gauss_seidel(A_test, b_test, tol=1e-8, max_iterations=5000)
    _, _, it_jac = jacobi(A_test, b_test, tol=1e-8, max_iterations=5000)
    
    iters_gs_list.append(it_gs)
    iters_jac_list.append(it_jac)

ax4.plot(sizes, iters_gs_list, 'b-o', linewidth=2, markersize=8, label='Gauss-Seidel')
ax4.plot(sizes, iters_jac_list, 'r--s', linewidth=2, markersize=8, label='Jacobi')
ax4.set_xlabel('System Size (n)', fontsize=12)
ax4.set_ylabel('Iterations to Converge', fontsize=12)
ax4.set_title('Iterations vs System Size', fontsize=14)
ax4.legend(fontsize=10)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved to 'plot.png'")

## Key Observations

1. **Convergence Speed**: Gauss-Seidel typically converges in approximately half the iterations of Jacobi for tridiagonal systems. This is because the spectral radius satisfies $\rho_{GS} \approx \rho_J^2$.

2. **Memory Efficiency**: Gauss-Seidel requires only one vector for storage (in-place updates), while Jacobi needs two vectors.

3. **Scalability**: Both methods show $O(n^2)$ iteration growth for tridiagonal systems, but Gauss-Seidel maintains its advantage at all scales.

4. **Parallelization**: Jacobi is naturally parallelizable (all updates independent), while Gauss-Seidel has sequential dependencies.

## Conclusion

The Gauss-Seidel method is a robust iterative solver that offers improved convergence over the Jacobi method for many linear systems. Its convergence is guaranteed for diagonally dominant and symmetric positive definite matrices, making it suitable for many scientific computing applications, particularly those arising from discretized partial differential equations.